# **Langchain in Chains: Google Search**
###Gerador de tweets automatizado com LLMs
---

## **Objetivo**
---
Vamos criar um gerador automático de tweets, utilizando como base um assunto qualquer de nossa escolha, para isso vamos utilizar o ***Google Serper API***, uma API que permite recuperar resultados de uma pesquisa no Google:

<center><img src='https://cdn.prod.website-files.com/659415b46df8ea43c3877776/65b7f02e3e78f985c9b5b108_serper-homepage.jpeg'></center>

Além disso vamos utilizar o LangChain para facilitar a implementação, juntamente com os LLMs Cohere para realizar embeddings e Gemini para geração dos tweets



## **Metodologia**
---
Seguimos os métodos presentes no artigo escolhido<sup>[1]</sup>, com algumas diferenças para poder utilizar LLMs gratuitos, já que o artigo utiliza OpenAI. Por conta dessa mudança de LLM algumas partes do código ficaram diferentes do artigo original, mas o resultado final é o mesmo, os passos seguidos são:

#### 1. Fazer o download das bibliotecas necessárias

#### 2. Setar as chaves de API que iremos utilizar
  
#### 3. Definir uma função que irá realizar a busca no Google, de acordo com um assunto passado como entrada

#### 4. Definir uma função que a partir dos resultados da busca retorna uma lista de 3 URLs mais relevantes

#### 5. Definir a função que irá separar os textos das URLs em blocos e realizar os embeddings, retornando um objeto ***vectorstore*** com FAISS

#### 6. Definir a função que irá gerar os tweets, fazendo uma busca por similaridade entre a query que fizemos no Google e o ***vectorstore*** criado no passo 5, juntando tudo em um único documento e criando os tweets utilizando a inteligência artificial



# **FAISS (Facebook AI Similarity Search)**
---
<center><img src='https://miro.medium.com/v2/resize:fit:720/format:webp/1*i7BwsiaZ71mCuq12enDJtA.jpeg'></center>

Como dito acima, vamos criar um ***vectorstore*** com FAISS para poder realizar uma busca por similaridade depois, vamos entender melhor o que é isso.

FAISS é uma biblioteca desenvolvida pela Meta AI, projetada especifamente para lidar com pesquisas de similaridade de forma eficiente, o que é útil para lidar com grandes conjuntos de dados.

Ele pode ser utilizado para criar um índice e realizar pesquisas com grande velocidade e eficiência de memória.

O FAISS agiliza as pesquisas de vizinhos mais próximos indexando vetores usando algoritmos sofisticados como:

1. **K-means clustering**: Esse algoritmo divide os dados em clusters, o que ajuda a restringir o espaço de pesquisa, concentrando-se nos clusters mais relevantes.

2. **Quantização de produto (PQ)**: O PQ comprime os vetores em códigos mais curtos, reduzindo significativamente o uso da memória e acelerando a pesquisa sem uma grande queda de precisão

3. **Quantização otimizada de produtos (OPQ)**: Uma versão aprimorada do PQ, o OPQ gira os dados para se ajustar melhor à grade de quantização, melhorando a precisação de vetores compactados.


### **Casos de uso do FAISS**
Por ser muito versátil e eficiente, o FAISS é ideal para uma variedade de aplicações de diferente tipos como:

- Sistemas de Recomendação
- Pesquisa de imagens e vídeos
- Detecção de anomalias
- Recuperação de informações (Utilizado neste projeto!)




# **Implementação**
---

## 1º passo:
Instalar as bibliotecas necessárias e importar o que vai ser utilizado:

In [98]:
!pip install -qU langchain-cohere
!pip install -qU langchain-google-genai
!pip install -qU unstructured
!pip install -qU faiss-cpu
!pip install -qU faiss-gpu
!pip install -qU langchain
!pip install -qU beautifulsoup4

In [230]:
import os
import json
import requests
import numpy as np
import faiss

from langchain_cohere import CohereEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.utilities import GoogleSerperAPIWrapper
from langchain import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_core.documents import Document
from bs4 import BeautifulSoup
from ast import literal_eval
from google.colab import userdata
from IPython.display import Markdown

## 2º passo:
Setar as chaves para utilizar os serviços das APIs, também definir o modelo que irá realizar os embeddings:

In [227]:
os.environ["COHERE_API_KEY"] = userdata.get("COHERE_API_KEY")
os.environ["SERPER_API_KEY"] = userdata.get("SERPER_API_KEY")
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

In [101]:
embeddings = CohereEmbeddings(model="embed-english-v3.0")

## 3º passo:

Definindo a função que realiza a busca baseado em um assunto de entrada:

In [142]:
# A função possui um parâmetro "query" que será a nossa busca desejada
def search(query: str) -> dict:

  # Instaciamos o objeto GoogleSerperAPIWrapper, que irá realizar a busca baseada na query
  # k será a quantidade de resultados, gl o país de origem das páginas e hl a língua
  search = GoogleSerperAPIWrapper(k=10, gl='br', hl='pt-br', type="search")
  response = search.results(query)

  # Por fim retornamos os resultados
  return response

In [232]:
# Aqui definimos nossa busca e utilizamos a função criada para
query = "Últimas notícias do futebol brasileiro"
search_results = search(query)
search_results

{'searchParameters': {'q': 'Últimas notícias do futebol brasileiro',
  'gl': 'br',
  'hl': 'pt-br',
  'type': 'search',
  'num': 10,
  'engine': 'google'},
 'organic': [{'title': 'Futebol | ge - Globo Esporte',
   'link': 'https://ge.globo.com/futebol/',
   'snippet': 'Acompanhe as notícias de Futebol no ge.globo.',
   'sitelinks': [{'title': 'Futebol internacional',
     'link': 'https://ge.globo.com/futebol/futebol-internacional/'},
    {'title': 'Brasileiro Feminino',
     'link': 'https://ge.globo.com/futebol/futebol-feminino/brasileiro-feminino/'},
    {'title': 'Futebol português',
     'link': 'https://ge.globo.com/futebol/futebol-internacional/futebol-portugues/'},
    {'title': 'Flamengo',
     'link': 'https://ge.globo.com/futebol/times/flamengo/'}],
   'position': 1},
  {'title': 'Futebol - UOL Esporte',
   'link': 'https://www.uol.com.br/esporte/futebol/ultimas/',
   'snippet': 'Acompanhe no UOL Esporte as Últimas notícias, fotos, vídeos, dicas e competições de futebol e pr

## 4º passo:

Definir uma função que, a partir dos resultados da pesquisa, retorna apenas 3 URLs que são consideradas mais relevantes:

In [228]:
# Passamos como parâmetro os resultados da pesquisa assim como a "query" que foi feita
def extract_top_article_urls(search_results: dict, query: str) -> list:

  # Primeiros tranformamos os resultados em uma string, para poder passar no template para Inteligência Artificial
  string = json.dumps(search_results)

  # Definimos o modelo que será utilizado para nossa tarefa
  modelo = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.8)

  # Escrevemos um template de prompt, ele irá analisar os resultados obtidos na nossa pesquisa e filtrar pelos 3 que ele considerar mais relevantes baseado na nossa pesquisa inicial, retornando uma lista de URLs
  prompt_template_str = """
  You are a world-class sports journalist with a keen eye for identifying content that drives views and interactions on Twitter. Your expertise lies in finding the most engaging, relevant, and timely sports articles.

  QUERY RESPONSE: {string}

  Above is the list of search results for the query "{query}".

  Your task is to meticulously select the top 3 articles from the list that will generate the highest engagement on Twitter. Return ONLY a LIST (ARRAY) of the URLs. Exclude any other information. Ensure the articles are fresh and not outdated. If the file or URL is invalid, return www.google.com.
  """

  # Criamos a cadeia que irá executar o template com os parâmetros necessários, juntamente com o modelo que será utilizado
  prompt_template = PromptTemplate(input_variables=["string", "query"], template=prompt_template_str)
  selection_chain =  prompt_template | modelo

  # Executamos a cadeia
  response = selection_chain.invoke({"string": string, "query": query})

  # Como a resposta que é retornada vem em uma String deste tipo " ```python["url1", "url2", "url3"]``` ", fazemos um tratamento para ficar no formato de lista de fato
  urls = response.content
  urls = urls.replace("`", "")
  urls = urls.replace("json", "")

  # Após o tratamento ele ainda é uma String, por isso utilizamos essa função para transformar uma String diretamente em Lista
  urls_list = literal_eval(urls)

  return urls_list

In [203]:
urls_list = extract_top_article_urls(search_results, query)
print(urls_list)

['https://ge.globo.com/futebol/', 'https://www.uol.com.br/esporte/futebol/ultimas/', 'https://www.lance.com.br/futebol-nacional']


## 5º passo:
Definir funções para extrair os textos das URLs e para separar os textos, criando o ***vectorstore***:

In [229]:
# Essa primeira função serve para recuperar o conteúdo das URLs
def fetch_text_from_url(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_5_0) AppleWebKit/536.1 (KHTML, like Gecko) Chrome/58.0.849.0 Safari/536.1'}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup.get_text()


# Toda essa próxima parte foi feita com auxílio do ChatGPT. Utilizando os embeddings do Cohere diversos erros aconteciam ao escrever o código igual ao do artigo, mais especificamente ao tentar instaciar o objeto FAISS.
# Por isso fomos implementando as sugestões feitas pela IA para tentar contornar os erros e obter o resultado esperado.

# Criamos uma classe que simula um armazenamento de documentos que é suportado pelo índice FAISS, oferecendo duas funções principais 'get' e 'search'
class DummyDocStore:
    def __init__(self, documents):
        self.documents = documents

    # Recupera um documento específico com base em seu ID, verificando se o ID está dentro dos limites da lista de documentos
    def get(self, doc_id):
        if 0 <= doc_id < len(self.documents):
            return self.documents[doc_id]
        else:
            raise ValueError(f"Document ID {doc_id} is out of bounds.")

    # Recupera documentos com base em uma lista de IDs.
    def search(self, doc_ids):

        # Verifica se 'doc_ids' é um inteiro (o que indica apenas um documento)
        if isinstance(doc_ids, int):
            return self.get(doc_ids)  # Retorna um único documento

        # Se 'doc_ids' for uma lista, retorna uma lista de documentos correspondente
        elif isinstance(doc_ids, list):
            return [self.get(doc_id) for doc_id in doc_ids]
        else:
            raise ValueError("doc_ids should be an int or list of ints.")


# Função para carregar e processar os conteúdos das URLs, dividir o texto em chunks, criar os embeddings e construir um índice FAISS para busca
def split_content_from_urls(urls_list: list) -> FAISS:

  # Adicionamos os textos brutos das URLs em uma lista
  texts = [fetch_text_from_url(url) for url in urls_list]

  # Criamos objetos 'Document' a partir dos textos
  documents = [Document(page_content=text) for text in texts]

  # Definimos o splitter que irá dividir os textos. Esse método faz o split de forma recursiva, e tenta manter 'pedaços' relacionados próximos um do outro
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

  # Iniciamos listas vazias que irão armazenar os chunks de texto e os IDs dos documentos originais
  split_documents = []
  doc_ids = []

  for i, doc in enumerate(documents):
      chunks = text_splitter.split_text(doc.page_content) # Divide o conteúdo de cada documento em chunks
      split_documents.extend(chunks) # Adiciona os chunks gerados à lista 'split_documents'
      doc_ids.extend([i] * len(chunks)) # Associa o ID do documento original para cada chunk gerado a partir dele. Isso permite mapear cada chunk de volta ao documento original

  # Cria embeddings para cada chunk de texto usando o modelo de embeddigns do Cohere
  chunk_embeddings = embeddings.embed_documents(split_documents)

  # Converte os embeddings para um array numpy de ponto flutuante, que é necessário para criar o índice FAISS
  chunk_embeddings = np.array(chunk_embeddings, dtype=np.float32)

  # Colocamos a dimensão dos nossos embeddings em uma variável (número de colunas)
  dim = chunk_embeddings.shape[1]

  # Criamos um índice FAISS utilizando a métrica de distância L2 (distância euclidiana) para comparação
  faiss_index = faiss.IndexFlatL2(dim)

  # Adicionamos os embeddings ao nosso índice FAISS
  faiss_index.add(chunk_embeddings)

  # Utilizamos a classe que criamos para armazenar os documentos originais
  docstore = DummyDocStore(documents)

  # Criamos um dicionário que mapeia os IDs dos chunks de texto para os IDs dos documentos originais
  index_to_docstore_id = {i: doc_ids[i] for i in range(len(split_documents))}

  # Instanciamos o objeto FAISS, com todos os parâmetros necessários
  faiss_store = FAISS(index=faiss_index, docstore=docstore, index_to_docstore_id=index_to_docstore_id, embedding_function=embeddings)

  return faiss_store

In [220]:
data = split_content_from_urls(urls_list)
print(type(data))

<class 'langchain_community.vectorstores.faiss.FAISS'>


## 6º passo:
Definir a função que irá gerar os tweets:

In [233]:
# Finalmente, definimos a função que irá gerar os tweets, baseado no índice FAISS para buscar documentos semelhantes, a 'query' que fizemos e o número 'k' de documentos semelhantes que desejamos recuperar
def generate_engaging_tweets(faiss_index: FAISS, query: str, k: int=3) -> str:

    # Busca os documentos mais semelhantes com a 'query' que foi feita
    similar_docs = faiss_index.similarity_search(query, k=k)

    # Verifica se 'similar_docs' é uma lista ou apenas um documento
    if isinstance(similar_docs, list):
        aggregated_content = " ".join([doc.page_content for doc in similar_docs if doc is not None])
    else:
        aggregated_content = similar_docs.page_content if similar_docs is not None else ""

    # Caso ocorra erro pelo tamanho do conteúdo, determine um limite para não ultrapassar a cota do LLM
    #if len(aggregated_content) > 10000:
      #aggregated_content = aggregated_content[:10000]


    # Definimos o modelo que será utilizado, o prompt, e criamos a chain que irá gerar os tweets
    modelo = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.8)

    prompt_template_str = """
        {docs}
        As a world-class sports journalist, you will summarize the text above to create engaging tweets around the topic "{query}".
        These tweets will be posted on Twitter to drive high engagement.
        Write them in brazilian portuguese.

        Please follow all of the following guidelines:
        1. Ensure the content is engaging and informative with good data.
        2. Keep the tweets concise, fitting within Twitter's character limit.
        3. Address the topic "{query}" very well and stay on point.
        4. Make sure the content is high quality and informative.
        5. Write in a way that is easy to read, digest, and understand.
        6. Provide actionable insights and advice, including resources and links if necessary.

        TWEET:
        """

    prompt_template = PromptTemplate(input_variables=["docs", "query"], template=prompt_template_str)

    tweet_summarizer_chain = prompt_template | modelo

    tweets = tweet_summarizer_chain.invoke({"docs":aggregated_content, "query":query})

    return tweets.content

In [234]:
tweet = generate_engaging_tweets(data, query)
Markdown(tweet)

## Últimas notícias do futebol brasileiro:

**Tweet 1:** 🚨 O Inter não perde para o Cruzeiro no Beira-Rio há mais de 10 anos! 😮 Será que o tabu se mantém? #Inter #Cruzeiro #Grenal #Brasileirão

**Tweet 2:** 🏥 Tite teve alta do hospital após internação por arritmia cardíaca. 🙏 Desejamos melhoras ao técnico! #Tite #SeleçãoBrasileira #ForçaTite

**Tweet 3:** 🔥 Flamengo faz proposta por Alex Sandro, ex-Juventus! 💰 Será que o Mengão vai repatriar o lateral? #Flamengo #AlexSandro #MercadoDaBola

**Tweet 4:** ⚽️  O Corinthians vendeu Wesley para a Arábia Saudita! 🇸🇦 O jogador será companheiro de Cristiano Ronaldo! #Corinthians #Wesley #CristianoRonaldo

**Tweet 5:** 🏆 Libertadores: cariocas complicam datas das quartas de final. 🤔 CBF busca alternativas! #Libertadores #Flamengo #Fluminense #CBF 

**Tweet 6:** 🏟️  Jogo do Fluminense hoje: saiba onde assistir, horário e escalações! ➡️ [link para o artigo] #Fluminense #Brasileirão #OndeAssistir

**Tweet 7:** 📊 Convocação para Seleção comprova acerto de Luiz Henrique ao escolher o Botafogo! #Botafogo #LuizHenrique #SeleçãoBrasileira

**Tweet 8:** 🚨 Santos terá novidade em todas as áreas do campo contra o Amazonas! #Santos #SérieB #FutebolBrasileiro


**Refêrencias:**

[1] Artigo Medium: https://medium.com/@okanyenigun/langchain-in-chains-28-google-search-af86874bd29c

[2] O que é Faiss (Facebook AI Similarity Search)?: https://www.datacamp.com/pt/blog/faiss-facebook-ai-similarity-search